Only the main process_and_describe function is shown.
The code below (generate_description_main.py) takes 3D models' downloaded files from the Thingiverse scraper and uses GenAI to generate:
1. a description using either provided image files or blender snapshots
2. A category for the model
3. tags for the model

This outputs a csv with the generated information for each model and can be paused at any time (since it costs money).
This can be skipped as there are around 40 models in the csv, but can be run if more want to be loaded.

In [5]:
# def process_and_describe(client, repo_owner, repo_name, local_extract_path, subdir, csv_writer, existing_ids, save_images=True):
#     """
#     Processes and describes each model in the repository.
#
#     Parameters:
#     - client (OpenAIClient): Instance of OpenAIClient.
#     - repo_owner (str): Owner of the repository.
#     - repo_name (str): Name of the repository.
#     - local_extract_path (str): Path where files are extracted.
#     - subdir (dict): Information about the subdirectory.
#     - csv_writer (csv.writer): CSV writer object.
#     - existing_ids (set): Set of already processed object IDs.
#     - save_images (bool): Whether to save processed images.
#     """
#     subdir_path = os.path.join(local_extract_path, subdir['path'])
#     os.makedirs(subdir_path, exist_ok=True)
#
#     try:
#         files = list_files_in_repo(repo_owner, repo_name, subdir['path'])
#     except Exception as e:
#         print(f"Error listing files in subdir {subdir['path']}: {e}")
#         return
#
#     if not files:
#         print("No files found in the repository.")
#         return
#
#     print(f"Found {len(files)} items in {subdir['path']} directory.")
#     for file_info in files:
#         if file_info['type'] == 'file' and file_info['name'].endswith('.zip'):
#             zip_path = os.path.join(subdir_path, file_info['name'])  # Path of zip file
#             try:
#                 download_file(file_info['download_url'], zip_path)  # Downloads file at path
#             except Exception as e:
#                 print(f"Skipping file due to download error: {zip_path} - {e}")
#                 continue
#
#             extract_zip(zip_path, subdir_path)  # Extracts zip to subdir
#             images_dir = os.path.join(subdir_path, 'images')
#             image_files = find_image_files(images_dir)  # Finds all images for each item
#
#             # Define the exclusion list
#             BROAD_TAGS_EXCLUSION = [
#                 "3d model", "fully assembled", "complete", "model", "assembly",
#                 "thing", "object", "product", "item", "prototype"
#             ]
#
#             if not image_files:
#                 print(f"No image files found in {zip_path}. Attempting to render using Blender.")
#                 # Locate the .stl or .obj file under 'files/' subdirectory
#                 files_subdir = os.path.join(subdir_path, 'files')
#                 if not os.path.isdir(files_subdir):
#                     print(f"No 'files/' directory found in {zip_path}. Skipping.")
#                     continue
#
#                 # Find all .stl and .obj files in 'files/' directory
#                 model_files = [f for f in os.listdir(files_subdir) if f.lower().endswith(('.stl', '.obj'))]
#                 if not model_files:
#                     print(f"No .stl or .obj files found in 'files/' directory of {zip_path}. Skipping.")
#                     continue
#
#                 # For simplicity, process the first .stl/.obj file found
#                 model_file = os.path.join(files_subdir, model_files[0])
#                 print(f"Found model file: {model_file}")
#
#                 # Define paths for Blender executable and script
#                 blender_executable = r"C:\Program Files\Blender Foundation\Blender 4.2\blender.exe"  # Update if different
#                 blender_script = ""
#                 rendered_image_path = os.path.join(subdir_path, 'rendered_image.png')
#
#                 # Call Blender to render the image
#                 rendered_image = render_model_with_blender(
#                     blender_executable,
#                     blender_script,
#                     model_file,
#                     rendered_image_path
#                 )
#
#                 if rendered_image and os.path.exists(rendered_image):
#                     image_to_use = rendered_image
#                 else:
#                     print(f"Failed to render image for {zip_path}. Skipping.")
#                     continue
#             else:
#                 # Use the largest image file if no combined image is found
#                 image_to_use = find_largest_image_file(image_files)
#                 if not image_to_use:
#                     print(f"No suitable image found in {zip_path}")
#                     continue
#
#             # Log the image being processed
#             print(f"Processing image: {image_to_use}")
#
#             # Extract keywords from the zip file name and add new keywords
#             zip_file_name = os.path.splitext(file_info['name'])[0]
#             keywords = zip_file_name.replace('-', ' ').replace('_', ' ').split()
#             keywords.extend(["complete", "assembled", "whole", "IMG_"])  # Keywords include name and these
#
#             # Generate initial guess based on the directory name
#             initial_guess = " ".join(keywords).capitalize()
#
#             # Extract object_id from subdir_path
#             id = os.path.basename(subdir_path)
#
#             # Check if object_id already exists
#             if id in existing_ids:
#                 print(f"Object ID {id} already exists in CSV. Skipping.")
#                 continue
#
#             # Generate description, category, and tags using OpenAI
#             ai_data = client.describe_image_model(image_to_use, initial_guess)
#             if ai_data:
#                 description = ai_data.get('description', '').strip()
#                 category = ai_data.get('category', '').strip()
#                 tags = ai_data.get('tags', [])
#
#                 if not description or not category or not tags:
#                     print(f"Incomplete AI data for {image_to_use}. Skipping.")
#                     continue
#
#                 # Filter out broad/generic tags
#                 filtered_tags = filter_tags(tags, BROAD_TAGS_EXCLUSION)
#
#                 # If no tags remain after filtering, assign a default tag
#                 if not filtered_tags:
#                     print(f"All tags for {image_to_use} were broad. Assigning default tag 'unknown'.")
#                     filtered_tags = ["unknown"]
#
#                 # Format tags as comma-separated string
#                 tags_str = ", ".join(filtered_tags)
#
#                 print(f"Description for {image_to_use}:\n{description}\n")
#                 print(f"Category: {category}")
#                 print(f"Tags: {tags_str}\n")
#
#                 # Derive name from the zip file name or any other logic
#                 name = " ".join(keywords).capitalize()
#
#                 # Write object_id, description, name, category, tags to CSV
#                 csv_writer.writerow([id, description, name, category, tags_str])
#                 print(f"Wrote data for Object ID {id} to CSV.\n")
#
#                 # Add to existing_ids to prevent reprocessing within the same run
#                 existing_ids.add(id)
#
#             # Save a copy of the image being processed for visual inspection if save_images is True
#             if save_images:
#                 try:
#                     output_dir = os.path.join(local_extract_path, 'processed_images')  # Puts them to processed_images
#                     os.makedirs(output_dir, exist_ok=True)
#                     img_output_path = os.path.join(output_dir, os.path.basename(image_to_use))  # Good for verifying correctness of GPT
#
#                     # Ensure path length is within acceptable limits
#                     if len(img_output_path) > 255:
#                         print(f"Path too long: {img_output_path}")
#                         continue
#
#                     # Save the image file
#                     with open(image_to_use, 'rb') as img_file:
#                         with open(img_output_path, 'wb') as output_file:
#                             output_file.write(img_file.read())
#                     print(f"Saved processed image to: {img_output_path}")
#                 except Exception as e:
#                     print(f"Error saving image {image_to_use}: {e}")
#
#             # Pause to manage load
#             time.sleep(1)

Next, this code uses the AI generated summaries above, as well as the nonAI summaries from the thingiverse Scraper. It constructs a knowledge graph with the following edges between each node: 
1. AI and nonAI summaries similarity using tf-idf vectors and cosine similarity calculation (labeled SIMILAR_SUMMARY_AI and SIMILAR_SUMMARY_NONAI)
2. AI and nonAI summaries similarity using BERT Embedding (labeled SIMILAR_EMBEDDING_AI and SIMILAR_EMBEDDING_NONAI)
3. Name similarity using tf-idf vectors and cosine similarity calculation (labeled SIMILAR_NAMES_ALL)
4. AI and nonAI tag similarity using Jaccard similarity (labeled SIMILAR_TAGS_AI and SIMILAR_TAGS_NONAI)

### Only the main function and output are shown.

Because there are much fewer AI description models than nonAI description models, right now only about 540 models have similarity edges based on AI calculations. However all models have nonAI elements and name edges between nodes. This creates a knoweldge graph with 7 edges, 4 of which are present in every single models and 3 of which are only present in about 40 models. However, with more descriptions generated, more AI based similarity edges can be created.

In [1]:
# def main(csv_ai_path, csv_nonai_path, k_threshold=True, k=1):
#     logging.info("Starting semantic network construction process.")
#
#     # Step 1: Connect to Neo4j
#     graph_db = connect_to_neo4j(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
#
#     # Step 2: Load Data
#     df_nonai, df_ai = load_data(csv_ai_path, csv_nonai_path)
#
#     # Step 3: Preprocess Data
#     df_nonai = preprocess_dataframe(df_nonai, is_ai=False)
#     df_ai = preprocess_dataframe(df_ai, is_ai=True)
#
#     # Step 3.1: Verification - Ensure all models have 'name'
#     num_nonai_with_name = df_nonai['name'].notna().sum()
#     num_ai_with_name = df_ai['name'].notna().sum()
#     total_models = df_nonai.shape[0] + df_ai.shape[0]
#
#     logging.info(f"Non-AI models with extracted names: {num_nonai_with_name}/{df_nonai.shape[0]}")
#     logging.info(f"AI models with names: {num_ai_with_name}/{df_ai.shape[0]}")
#     logging.info(f"Total models with names: {num_nonai_with_name + num_ai_with_name}/{total_models}")
#
#     if (num_nonai_with_name + num_ai_with_name) < total_models:
#         logging.warning("Some models do not have a name extracted. They will not be included in 'SIMILAR_NAMES' relationships.")
#
#     # Step 4: Extract Key Components
#     df_nonai = extract_key_nouns_adjectives(df_nonai, is_ai=False)
#     df_ai = extract_key_nouns_adjectives(df_ai, is_ai=True)
#
#     # Step 5: Expand Synonyms
#     df_nonai, synonym_dict_nonai = expand_synonyms(df_nonai, is_ai=False)
#     df_ai, synonym_dict_ai = expand_synonyms(df_ai, is_ai=True)
#
#     # Step 6: Perform Named Entity Recognition
#     df_nonai = perform_ner(df_nonai, is_ai=False)
#     df_ai = perform_ner(df_ai, is_ai=True)
#
#     # Step 7: Weight primary nouns and named entities, incorporating synonyms in descriptions
#     df_nonai = duplicate_primary_nouns(df_nonai, is_ai=False, weight=2.0)
#     df_ai = duplicate_primary_nouns(df_ai, is_ai=True, weight=2.0)
#
#     # Step 8: Text Similarity Calculation
#     logging.info("Calculating similarity metrics...")
#
#     # Non-AI Similarities
#     tfidf_nonai, vectorizer_nonai = compute_tfidf_vectors(df_nonai, description_type='nonai')
#     cosine_sim_nonai = compute_cosine_similarity_matrix(tfidf_nonai)
#     embedding_sim_nonai = compute_sentence_transformer_similarity(df_nonai, description_type='nonai')
#
#     # AI Similarities (only among AI models)
#     if not df_ai.empty and df_ai['clean_description_ai'].str.strip().any():
#         tfidf_ai, vectorizer_ai = compute_tfidf_vectors(df_ai, description_type='ai')
#         cosine_sim_ai = compute_cosine_similarity_matrix(tfidf_ai)
#         embedding_sim_ai = compute_sentence_transformer_similarity(df_ai, description_type='ai')
#     else:
#         logging.warning("No AI descriptions available. Skipping AI similarity computations.")
#         cosine_sim_ai = np.array([])
#         embedding_sim_ai = np.array([])
#
#     # Compute tag similarity for Non-AI models
#     tag_sim_nonai = compute_tag_similarity(df_nonai, is_ai=False)
#
#     # Compute tag similarity for AI models
#     if not df_ai.empty:
#         tag_sim_ai = compute_tag_similarity(df_ai, is_ai=True)
#     else:
#         tag_sim_ai = np.array([])
#
#     # Compute name similarity for all models (both AI and Non-AI)
#     logging.info("Computing name similarity for all models...")
#     combined_df_names = pd.concat([
#         df_nonai[['id', 'name']],
#         df_ai[['id', 'name']]
#     ], ignore_index=True)
#
#     # Remove models without a name
#     combined_df_names = combined_df_names[combined_df_names['name'].str.strip() != ''].reset_index(drop=True)
#
#     if not combined_df_names.empty:
#         cosine_sim_name = compute_names_similarity_matrix(combined_df_names['name'].tolist())
#     else:
#         cosine_sim_name = np.array([])
#         logging.warning("No names available to compute name similarity.")
#
#     # Similarity matrices
#     similarity_matrices_nonai = {
#         "SIMILAR_SUMMARY_NONAI": cosine_sim_nonai,
#         "SIMILAR_TAGS_NONAI": tag_sim_nonai,
#         "SIMILAR_EMBEDDING_NONAI": embedding_sim_nonai
#     }
#
#     similarity_matrices_ai = {}
#     if not df_ai.empty and df_ai['clean_description_ai'].str.strip().any():
#         similarity_matrices_ai = {
#             "SIMILAR_SUMMARY_AI": cosine_sim_ai,
#             "SIMILAR_TAGS_AI": tag_sim_ai,
#             "SIMILAR_EMBEDDING_AI": embedding_sim_ai
#         }
#     else:
#         logging.warning("AI similarity matrices are empty.")
#
#     # Add name similarity for all models if available
#     if cosine_sim_name.size > 0:
#         similarity_matrices_nonai["SIMILAR_NAMES_ALL"] = cosine_sim_name
#
#     # Determine dynamic thresholds if enabled
#     thresholds_nonai = {}
#     thresholds_ai = {}
#     thresholds_all_names = {}
#     if k_threshold:
#         for rel_type, sim_matrix in similarity_matrices_nonai.items():
#             if sim_matrix.size > 0:
#                 threshold = determine_threshold_mean_std(sim_matrix, k=k)
#                 thresholds_nonai[rel_type] = threshold
#
#         for rel_type, sim_matrix in similarity_matrices_ai.items():
#             if sim_matrix.size > 0:
#                 threshold = determine_threshold_mean_std(sim_matrix, k=k)
#                 thresholds_ai[rel_type] = threshold
#
#         if "SIMILAR_NAMES_ALL" in similarity_matrices_nonai:
#             threshold = determine_threshold_mean_std(similarity_matrices_nonai["SIMILAR_NAMES_ALL"], k=k)
#             thresholds_nonai["SIMILAR_NAMES_ALL"] = threshold
#     else:
#         thresholds_nonai = {rel_type: 0.0 for rel_type in similarity_matrices_nonai.keys()}
#         thresholds_ai = {rel_type: 0.0 for rel_type in similarity_matrices_ai.keys()}
#         if "SIMILAR_NAMES_ALL" in similarity_matrices_nonai:
#             thresholds_nonai["SIMILAR_NAMES_ALL"] = 0.0
#
#     # Visualize similarity distribution with threshold
#     # for rel_type, sim_matrix in similarity_matrices_nonai.items():
#     #     threshold = thresholds_nonai.get(rel_type, None) if k_threshold else None
#     #     visualize_similarity_distribution(
#     #         sim_matrix,
#     #         relationship_type=rel_type,
#     #         threshold=threshold
#     #     )
#     #
#     # for rel_type, sim_matrix in similarity_matrices_ai.items():
#     #     threshold = thresholds_ai.get(rel_type, None) if k_threshold else None
#     #     visualize_similarity_distribution(
#     #         sim_matrix,
#     #         relationship_type=rel_type,
#     #         threshold=threshold
#     #     )
#
#     # Step 9: Knowledge Graph Construction with Embeddings
#     logging.info("Constructing knowledge graph with embeddings...")
#     create_nodes_with_embeddings(graph_db, df_nonai, is_ai=False)
#     create_nodes_with_embeddings(graph_db, df_ai, is_ai=True)
#
#     # Step 10: Create Edges Based on Similarity
#     create_edges_batch(
#         graph_db,
#         df_nonai,
#         similarity_matrices_nonai,
#         thresholds_nonai,
#         is_ai=False,
#         batch_size=1000
#     )
#
#     create_edges_batch(
#         graph_db,
#         df_ai,
#         similarity_matrices_ai,
#         thresholds_ai,
#         is_ai=True,
#         batch_size=1000
#     )
#
#     # Step 11: Additional Optimizations
#     # Define all relationship types for pruning
#     relationship_types_nonai = list(similarity_matrices_nonai.keys())
#     relationship_types_ai = list(similarity_matrices_ai.keys())
#
#     # Prune edges below the dynamic threshold for each relationship type
#     graph_pruning(
#         graph_db,
#         relationship_types_nonai + relationship_types_ai,
#         thresholds={**thresholds_nonai, **thresholds_ai}
#     )
#
#     # Perform clustering for each similarity type
#     for rel_type in similarity_matrices_nonai.keys():
#         description_type = rel_type.lower().replace("similar_", "")
#         similarity_matrix = similarity_matrices_nonai[rel_type]
#         threshold = thresholds_nonai.get(rel_type, 0.0)
#         node_clustering(
#             graph_db,
#             df_nonai,
#             similarity_matrix,
#             threshold=threshold,
#             description_type=description_type
#         )
#
#     for rel_type in similarity_matrices_ai.keys():
#         description_type = rel_type.lower().replace("similar_", "")
#         similarity_matrix = similarity_matrices_ai[rel_type]
#         threshold = thresholds_ai.get(rel_type, 0.0)
#         node_clustering(
#             graph_db,
#             df_ai,
#             similarity_matrix,
#             threshold=threshold,
#             description_type=description_type
#         )
#
#     # Step 12: Create Similar Names Relationships for All Models
#     logging.info("Creating 'SIMILAR_NAMES' relationships based on extracted names...")
#
#     # Combine AI and Non-AI data for names
#     combined_df = pd.concat([
#         df_nonai[['id', 'name']],
#         df_ai[['id', 'name']]
#     ], ignore_index=True)
#
#     # Remove duplicates based on name
#     name_groups = combined_df.groupby('name')['id'].apply(list).reset_index()
#
#     # Create 'similar_names' relationships where models share the same name
#     similar_names_relationships = []
#     for _, row in tqdm(name_groups.iterrows(), total=name_groups.shape[0], desc="Processing Similar Names"):
#         model_ids = row['id']
#         if len(model_ids) > 1 and row['name']:
#             for combo in combinations(model_ids, 2):
#                 similar_names_relationships.append({
#                     "source_id": combo[0],
#                     "target_id": combo[1],
#                     "score": 1.0  # Assuming exact match
#                 })
#
#     # Insert 'SIMILAR_NAMES' relationships in batches
#     batch_size = 1000
#     for i in tqdm(range(0, len(similar_names_relationships), batch_size), desc="Inserting 'SIMILAR_NAMES' Relationships"):
#         batch = similar_names_relationships[i:i + batch_size]
#         merge_relationships_batch(graph_db, batch, "SIMILAR_NAMES")
#
#     logging.info(f"Inserted total 'SIMILAR_NAMES' relationships: {len(similar_names_relationships)}")
#
#     logging.info("Knowledge graph construction complete.")
#
#
# if __name__ == "__main__":
#     # Set k_threshold=True to enable dynamic thresholding
#     # Set k_threshold=False to disable dynamic thresholding
#     # Adjust k as needed (e.g., k=1 for mean + 1*std)
#     main(
#         csv_ai_path=CSV_AI_PATH,
#         csv_nonai_path=CSV_NONAI_PATH,
#         k_threshold=True,
#         k=2,
#     )

C:\Users\noski\PycharmProjects\Semantic_Search_Thingiverse\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\noski\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\noski\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\noski\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\noski\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-11-26 13:17:01,192 [INFO] SpaCy Pipeline 

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

2024-11-26 13:18:08,146 [INFO] Completed Sentence-BERT similarity computation.
2024-11-26 13:18:08,147 [INFO] Computing TF-IDF vectors on weighted descriptions (ai)...
2024-11-26 13:18:08,218 [INFO] Completed TF-IDF computation.
2024-11-26 13:18:08,219 [INFO] Computing cosine similarity matrix...
2024-11-26 13:18:08,235 [INFO] Completed cosine similarity computation.
2024-11-26 13:18:08,237 [INFO] Computing Sentence-BERT similarity (ai)...
2024-11-26 13:18:08,242 [INFO] Use pytorch device_name: cpu
2024-11-26 13:18:08,243 [INFO] Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/17 [00:00<?, ?it/s]

2024-11-26 13:18:19,051 [INFO] Completed Sentence-BERT similarity computation.
2024-11-26 13:18:19,052 [INFO] Computing tag similarity matrices using Jaccard similarity for Non-AI models...
Computing Tag Similarity: 100%|██████████| 477753/477753 [00:00<00:00, 695668.57it/s]
2024-11-26 13:18:19,745 [INFO] Completed tag similarity computation.
2024-11-26 13:18:19,746 [INFO] Computing tag similarity matrices using Jaccard similarity for AI models...
Computing Tag Similarity: 100%|██████████| 144991/144991 [00:00<00:00, 784010.20it/s]
2024-11-26 13:18:19,937 [INFO] Completed tag similarity computation.
2024-11-26 13:18:19,937 [INFO] Computing name similarity for all models...
2024-11-26 13:18:19,940 [INFO] Computing name similarity using TF-IDF and cosine similarity...
2024-11-26 13:18:19,961 [INFO] Completed name similarity computation.
2024-11-26 13:18:19,963 [INFO] Determining threshold based on mean + 2*std of similarity scores...
2024-11-26 13:18:19,974 [INFO] Threshold set at mean (

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Merging Nodes in Batches: 100%|██████████| 1/1 [00:04<00:00,  4.43s/it]
2024-11-26 13:18:43,448 [INFO] Completed creating Non-AI nodes with embeddings in Neo4j.
2024-11-26 13:18:43,452 [INFO] Creating AI nodes in Neo4j with embeddings...
2024-11-26 13:18:43,455 [INFO] Use pytorch device_name: cpu
2024-11-26 13:18:43,457 [INFO] Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/17 [00:00<?, ?it/s]

Merging Nodes in Batches: 100%|██████████| 1/1 [00:01<00:00,  1.83s/it]
2024-11-26 13:18:55,672 [INFO] Completed creating AI nodes with embeddings in Neo4j.
2024-11-26 13:18:55,676 [INFO] Creating Non-AI similarity edges in Neo4j in batches...
2024-11-26 13:18:55,678 [INFO] Processing Non-AI relationship type: SIMILAR_SUMMARY_NONAI
Processing Pairs for SIMILAR_SUMMARY_NONAI: 100%|██████████| 477753/477753 [12:40<00:00, 627.81it/s]
2024-11-26 13:32:01,577 [INFO] Inserted final batch of 548 relationships for SIMILAR_SUMMARY_NONAI.
2024-11-26 13:32:01,578 [INFO] Total edges created for SIMILAR_SUMMARY_NONAI: 16548
2024-11-26 13:32:01,579 [INFO] Processing Non-AI relationship type: SIMILAR_TAGS_NONAI
Processing Pairs for SIMILAR_TAGS_NONAI: 100%|██████████| 477753/477753 [14:17<00:00, 557.06it/s]
2024-11-26 13:46:23,446 [INFO] Inserted final batch of 88 relationships for SIMILAR_TAGS_NONAI.
2024-11-26 13:46:23,448 [INFO] Total edges created for SIMILAR_TAGS_NONAI: 18088
2024-11-26 13:46:23

Lastly, this is the user search interface code for this combined system (model_search_by_sim_type.py). This allows the user to pick which edges they want to traverse:
'1': 'SIMILAR_TAGS_NONAI',
'2': 'SIMILAR_TAGS_AI',
'3': 'SIMILAR_SUMMARY_NONAI',
'4': 'SIMILAR_SUMMARY_AI',
'5': 'SIMILAR_EMBEDDING_NONAI',
'6': 'SIMILAR_EMBEDDING_AI',
'7': 'SIMILAR_NAMES'

### Only the main function and output are shown.

The user can pick one of these or multiple and weigh them by importance. Then, advanced functions like filtering, picking, and doing further searches are included, just like in the model_search.py code.


In [ ]:
# def main():
#     """
#     Main function to handle user queries and perform search on the knowledge graph using multiple similarity edges.
#     """
#     print("Initializing Model Search...")
#
#     # Connect to Neo4j
#     graph_db = connect_to_neo4j(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
#
#     # Load Sentence Transformer model once
#     logging.info("Loading SentenceTransformer model...")
#     model = SentenceTransformer('all-MiniLM-L6-v2')
#     logging.info("SentenceTransformer model loaded successfully.")
#
#     # Fetch all models with embeddings and precompute summary embeddings
#     df_models = fetch_all_models(graph_db, model)
#
#     while True:
#         # Prompt user for input
#         user_query = input("\nEnter your search query (or type 'exit' to quit): ").strip()
#         if user_query.lower() == 'exit':
#             print("Exiting Model Search. Goodbye!")
#             break
#         if not user_query:
#             print("Empty query. Please enter a valid search term.")
#             continue
#
#         # Prompt user to select similarity types
#         selected_similarity_types = prompt_similarity_types()
#
#         # Prompt user to assign weights to the selected similarity types
#         similarity_weights = prompt_similarity_weights(selected_similarity_types)
#
#         preprocessed_query = user_query  # No preprocessing needed
#         if not preprocessed_query.strip():
#             print("Invalid query. Please enter a non-empty search term.")
#             continue
#
#         # Extract tags from the query (assuming tags are the main nouns)
#         doc = nlp(user_query)
#         query_tags = set([token.text.lower() for token in doc if token.pos_ == 'NOUN'])
#
#         # Aggregate similarity scores
#         logging.info("Aggregating similarity scores...")
#         df_aggregated = aggregate_similarities(
#             df_models,
#             similarity_weights,
#             selected_similarity_types,
#             query_tags,
#             preprocessed_query,
#             model
#         )
#
#         if df_aggregated.empty:
#             print("No similar models found based on the selected similarity types.")
#             continue
#
#         # Display the top-k results
#         top_k = 10  # Number of top results to retrieve
#         top_results = display_results(df_aggregated, top_k=top_k)
#
#         if top_results is None or top_results.empty:
#             print("No similar models to display.")
#             continue
#
#         # Initialize the current search scope with the initial top results
#         current_scope = top_results
#
#         while True:
#             # Prompt user to apply further filters or select a model
#             selected = prompt_filter(current_scope)
#             if selected == 'yes':
#                 # User wants to filter further
#                 # Prompt to select a model from current_scope
#                 try:
#                     selection = int(input(
#                         f"Select a model to view similar models by entering its rank (1-{len(current_scope)}): ").strip())
#                     if 1 <= selection <= len(current_scope):
#                         selected_model_id = current_scope.iloc[selection - 1]['id']
#                         logging.info(f"Selected Model ID for further filtering: {selected_model_id}")
#                     else:
#                         print(f"Please enter a number between 1 and {len(current_scope)}.")
#                         continue
#                 except ValueError:
#                     print("Invalid input. Please enter a valid number.")
#                     continue
#
#                 # Fetch model details
#                 selected_details = fetch_model_details(graph_db, selected_model_id)
#                 if not selected_details:
#                     print(f"No model found with ID: {selected_model_id}.")
#                     continue
#
#                 # Prompt user to select similarity types for further filtering
#                 print("\nFor further filtering, you can select similarity types again.")
#                 further_similarity_types = prompt_similarity_types()
#
#                 # Prompt user to assign weights to the selected similarity types
#                 further_similarity_weights = prompt_similarity_weights(further_similarity_types)
#
#                 # Extract tags from the selected model's name and descriptions
#                 selected_model_tags = set()
#                 if pd.notnull(selected_details['tags_nonai']):
#                     selected_model_tags.update(
#                         [tag.strip().lower() for tag in selected_details['tags_nonai'].split(',') if tag.strip()])
#                 if pd.notnull(selected_details['tags_ai']):
#                     selected_model_tags.update([tag.strip().lower() for tag in selected_details['tags_ai'].split(',') if tag.strip()])
#
#                 # Since descriptions are preprocessed, use them directly
#                 selected_model_summary = selected_details['description_nonai'] if pd.notnull(
#                     selected_details['description_nonai']) else selected_details['description_ai']
#
#                 # Aggregate similarity scores based on the selected model's attributes
#                 logging.info("Aggregating similarity scores for further filtering...")
#                 df_new_similar = aggregate_similarities(
#                     df_models,
#                     further_similarity_weights,
#                     further_similarity_types,
#                     selected_model_tags,
#                     selected_model_summary,
#                     model
#                 )
#
#                 if df_new_similar.empty:
#                     print("No further models found based on your selection.")
#                     break  # Exit filtering loop
#
#                 # Display the new set of similar models
#                 print(f"\nModels similar to Model ID: {selected_model_id}:")
#                 similar_top_k = 10  # Number of top similar models to display
#                 similar_top_results = display_results(df_new_similar, top_k=similar_top_k)
#
#                 if similar_top_results is None or similar_top_results.empty:
#                     print("No similar models found in this step.")
#                     break  # Exit filtering loop
#
#                 # Update the current scope to the new similar models
#                 current_scope = similar_top_results
#
#             elif selected == 'no':
#                 # User does not want to filter further
#                 break  # Exit filtering loop
#             else:
#                 # User selected a model by rank to view its full summary
#                 selected_model_id = selected
#                 # Fetch the full details of the selected model
#                 selected_details = fetch_model_details(graph_db, selected_model_id)
#                 if selected_details:
#                     print(f"\n--- Full Summary of Selected Model ---")
#                     print(f"ID: {selected_details['id']}")
#                     print(f"Name: {selected_details['name'] if pd.notnull(selected_details['name']) else 'N/A'}")
#                     print(
#                         f"Tags Non-AI: {selected_details['tags_nonai'] if pd.notnull(selected_details['tags_nonai']) else 'N/A'}")
#                     print(
#                         f"Tags AI: {selected_details['tags_ai'] if pd.notnull(selected_details['tags_ai']) else 'N/A'}")
#                     print(
#                         f"Description Non-AI: {selected_details['description_nonai'] if pd.notnull(selected_details['description_nonai']) else 'N/A'}")
#                     print(
#                         f"Description AI: {selected_details['description_ai'] if pd.notnull(selected_details['description_ai']) else 'N/A'}")
#                     print(f"---------------------------------------\n")
#                 else:
#                     print(f"No details found for Model ID: {selected_model_id}.")
#
#                 # After displaying the summary, ask if the user wants another search
#                 while True:
#                     another_search = input("Do you want to perform another search? (yes/no): ").strip().lower()
#                     if another_search in ['yes', 'y']:
#                         break  # Break to the main search loop
#                     elif another_search in ['no', 'n']:
#                         print("Exiting Model Search. Goodbye!")
#                         sys.exit(0)
#                     else:
#                         print("Please answer with 'yes' or 'no'.")
#                 break  # Exit filtering loop to perform another search
#
# if __name__ == "__main__":
#     main()
